In [1]:
#Using this nb to clean my test csv of nulls so..

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
ames = pd.read_csv('../data/test.csv')

In [24]:
ames.head()
pd.set_option('display.max_rows', None)

In [4]:
#Same cleaning as training data. Copy and paste
lf_mean = ames['Lot Frontage'].mean()
ames['Lot Frontage']= ames['Lot Frontage'].fillna(round(lf_mean))
ames['Alley'] = ames['Alley'].fillna('None')
ames['Mas Vnr Type'] = ames['Mas Vnr Type'].fillna('None')
ames['Mas Vnr Area']=ames['Mas Vnr Area'].fillna(0)
ames['Bsmt Qual'] = ames['Bsmt Qual'].fillna("None")
ames['Bsmt Cond'] = ames['Bsmt Cond'].fillna('None')
ames['Bsmt Exposure'] =ames['Bsmt Exposure'].fillna('No')
ames['BsmtFin Type 1'] =ames['BsmtFin Type 1'].fillna("Unf")
ames['BsmtFin Type 2'] =ames['BsmtFin Type 2'].fillna("Unf")
ames['BsmtFin SF 1']= ames['BsmtFin SF 1'].fillna(0) 
ames['BsmtFin SF 2'] = ames['BsmtFin SF 2'].fillna(0)
ames['Bsmt Unf SF'] = ames['Bsmt Unf SF'].fillna(0)
ames['Total Bsmt SF'] = ames['Total Bsmt SF'].fillna(0)
ames['Bsmt Full Bath'] = ames['Bsmt Full Bath'].fillna(0)
ames['Bsmt Half Bath'] = ames['Bsmt Half Bath'].fillna(0)
ames['Garage Cars'] = ames['Garage Cars'].fillna(0)           
ames['Garage Area'] = ames['Garage Area'].fillna(0)
ames['Fireplace Qu'] =  ames['Fireplace Qu'].fillna('No')
ames['Pool QC'] = ames['Pool QC'].fillna('No')
ames['Fence'] = ames['Fence'].fillna('No')
ames['Misc Feature'] = ames['Misc Feature'].fillna('No')
ames['Garage Finish'] = ames['Garage Finish'].fillna('None')
ames['Garage Qual'] = ames['Garage Finish'].fillna('None')
ames['Garage Cond'] = ames['Garage Cond'].fillna('None')
ames['Garage Type'] = ames['Garage Type'].fillna('None')
ames['Garage Cars'] = ames['Garage Cars'].fillna(0)
ames['Garage Area'] = ames['Garage Area'].fillna(0)
y_built = ames['Garage Yr Blt'].mean()
ames['Garage Yr Blt'] = ames['Garage Yr Blt'].fillna(round(y_built))

In [5]:
ames.isnull().sum()

Id              0
PID             0
MS SubClass     0
MS Zoning       0
Lot Frontage    0
               ..
Misc Feature    0
Misc Val        0
Mo Sold         0
Yr Sold         0
Sale Type       0
Length: 80, dtype: int64

In [6]:
ames.columns = ames.columns.str.replace(" ", "_")
ames.columns = ames.columns.str.lower()

In [7]:
clean = ames[['id', 'pid', 'ms_subclass', 'ms_zoning', 'lot_frontage', 'lot_area',
       'lot_shape', 'land_contour', 'utilities', 'land_slope', 'overall_qual',
       'overall_cond', 'year_built', 'year_remod/add', 'mas_vnr_area',
       'exter_qual', 'exter_cond', 'bsmt_qual', 'bsmt_cond', 'bsmt_exposure',
       'bsmtfin_type_1', 'bsmtfin_sf_1', 'bsmtfin_type_2', 'bsmtfin_sf_2',
       'bsmt_unf_sf', 'total_bsmt_sf', 'heating_qc', 'central_air',
       'electrical', '1st_flr_sf', '2nd_flr_sf', 'low_qual_fin_sf',
       'gr_liv_area', 'bsmt_full_bath', 'bsmt_half_bath', 'full_bath',
       'half_bath', 'bedroom_abvgr', 'kitchen_abvgr', 'kitchen_qual',
       'totrms_abvgrd', 'functional', 'fireplaces', 'fireplace_qu',
       'garage_yr_blt', 'garage_finish', 'garage_cars', 'garage_area',
       'garage_qual', 'garage_cond', 'paved_drive', 'wood_deck_sf',
       'open_porch_sf', 'enclosed_porch', '3ssn_porch', 'screen_porch',
       'pool_area', 'pool_qc', 'fence', 'misc_val', 'mo_sold', 'yr_sold'
       ]]

In [8]:
grdex = np.where(ames['gr_liv_area']>4000)
ames = ames.drop(ames.index[grdex])

In [9]:
fdex=np.where(ames['fireplaces']>2.9)
ames = ames.drop(ames.index[fdex])

In [10]:
#Getting the test set ready for modeling. Copy and paste for the most part
ordinal_change = ames[['lot_shape','utilities','land_slope','exter_qual','exter_cond','bsmt_qual','bsmt_cond',
                       'bsmt_exposure', 'bsmtfin_type_1','bsmtfin_type_2',
                       'heating_qc','electrical','kitchen_qual','functional','fireplace_qu','garage_finish',
                        'garage_qual','garage_cond','paved_drive','pool_qc','fence','central_air']]

#lot shape
lot_s_o = {'Reg':4, 'IR1':3,'IR2':2,'IR3':1}

#Utilities
util_o = {'AllPub':4,'NoSewr':3,'NoSeWa':2,'ELO':1}

#land_slope
land_s_o = {'Gtl':3,'Mod':2,'Sev':1}

#bsmt_fin1/2
bsmt_o = {'GLQ':6,'ALQ':5,'BLQ':4,'Rec':3,'LwQ':2,'Unf':1 }
          
#bsmt_exposure
bs_e_o = {'Av': 3,'Mn': 2,'No': 1}
          

#functional-this one was a little difficult to construct
func_o = {'Typ':4,'Min1':3,'Min2':2,'Mod':2,'Maj1':1,'Maj2':1,'Sev':0,'Sal':0}

#paved drive
p_d_o = {'Y' : 2,'P' : 1,'N' : 0}
          
#fence
fence_o = {'GdPrv':4,'MnPrv':3,'GdWo':2,'MnWw':1}

#Electrical
elec_o = {'SBrkr':4,'FuseA':3,'FuseF':2,'FuseP':1,'Mix':0}

#garage_finish
ga_f_o = {'Fin': 3,'RFn': 2,'Unf':1}

#central air(non ordinal but needs numeric)
c_o = {'N':0, 'Y':1}

#Getting to the other columns. They have the same scale
all_o = {'Ex': 5,'Gd': 4,'TA': 3,'Fa': 2,'Po': 1,'None': 0}

full_ordinal = {**all_o, **c_o, **ga_f_o, **elec_o, **fence_o, **p_d_o, **func_o,
                **bs_e_o, **bsmt_o, **land_s_o, **util_o, **lot_s_o}
                
for i in ordinal_change.columns:
    ames = ames.replace({i: full_ordinal})

In [12]:
zone = ['A','C','I', 'FV', 'C (all)', 'A (agr)', 'I (all)']
ames = ames[ames.ms_zoning.isin(zone)== False]

ames['ms_zoning'] = ames['ms_zoning'].map({'RH': 4, 'RL':3, "RP":2, 'RM':1})
ames['land_contour'] = ames['land_contour'].map({'HLS': 4, 'Bnk':3, "Lvl":2, 'Low':1})

In [ ]:
ames = ames.drop(columns = ['lot_shape','fence','kitchen_abvgr','enclosed_porch','overall_cond',
                           'ms_subclass','land_slope','bsmt_half_bath','low_qual_fin_sf','yr_sold',
                           'bsmtfin_type_2','misc_val'])

In [29]:
ames.shape

(825, 79)

In [17]:
ames = ames.drop(columns = 'pid')


KeyError: "['pid'] not found in axis"

In [19]:
test_final = ames[['id','ms_subclass', 'ms_zoning', 'lot_frontage', 'lot_area',
       'lot_shape', 'land_contour', 'utilities', 'land_slope', 'overall_qual',
       'overall_cond', 'year_built', 'year_remod/add', 'mas_vnr_area',
       'exter_qual', 'exter_cond', 'bsmt_qual', 'bsmt_cond', 'bsmt_exposure',
       'bsmtfin_type_1', 'bsmtfin_sf_1', 'bsmtfin_type_2', 'bsmtfin_sf_2',
       'bsmt_unf_sf', 'total_bsmt_sf', 'heating_qc', 'central_air',
       'electrical', '1st_flr_sf', '2nd_flr_sf', 'low_qual_fin_sf',
       'gr_liv_area', 'bsmt_full_bath', 'bsmt_half_bath', 'full_bath',
       'half_bath', 'bedroom_abvgr', 'kitchen_abvgr', 'kitchen_qual',
       'totrms_abvgrd', 'functional', 'fireplaces', 'fireplace_qu',
       'garage_yr_blt', 'garage_finish', 'garage_cars', 'garage_area',
       'garage_qual', 'garage_cond', 'paved_drive', 'wood_deck_sf',
       'open_porch_sf', 'enclosed_porch', '3ssn_porch', 'screen_porch',
       'pool_area', 'pool_qc', 'fence', 'misc_val', 'mo_sold', 'yr_sold'
       ]]

In [21]:
test_final.head()

,id,ms_subclass,ms_zoning,lot_frontage,lot_area,lot_shape,land_contour,utilities,land_slope,overall_qual,...,open_porch_sf,enclosed_porch,3ssn_porch,screen_porch,pool_area,pool_qc,fence,misc_val,mo_sold,yr_sold
0,2658,190,1,69.0,9142,4,2,4,3,6,...,60,112,0,0,0,1,1,0,4,2006
1,2718,90,3,70.0,9662,3,2,4,3,5,...,0,0,0,0,0,1,1,0,8,2006
2,2414,60,3,58.0,17104,3,2,4,3,7,...,24,0,0,0,0,1,1,0,9,2006
3,1989,30,1,60.0,8520,4,2,4,3,5,...,0,184,0,0,0,1,1,0,7,2007
4,625,20,3,70.0,9500,3,2,4,3,6,...,76,0,0,185,0,1,1,0,7,2009


In [28]:
test_final.shape

(825, 61)

In [27]:
test_final.to_csv('../data/final_test.csv')